In [132]:
import os.path as osp
import pandas as pd
import networkx as nx
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.datasets import Planetoid, Coauthor, Amazon
from torch_geometric.utils import train_test_split_edges, dense_to_sparse
from torch_geometric.nn import GAE, VGAE, APPNP
import torch_geometric.transforms as T
from torch_geometric.data import Data

In [133]:
path = "hw2_data/"
dss = ['dataset1','dataset2','dataset3'] #datasets
datasets = dict()
for ds in dss:
    datasets[ds] = dict()
    datasets[ds]['content'] = pd.read_csv(path+ds+"/content.csv",delimiter = '\t',header = None)
    datasets[ds]['train'] = pd.read_csv(path+ds+"/train.csv",delimiter = ',')
    datasets[ds]['test'] = pd.read_csv(path+ds+"/test.csv",delimiter = ',')
    datasets[ds]['upload'] = pd.read_csv(path+ds+"/upload.csv",delimiter = ',')
datasets[dss[2]]['test'].head()

,id,to,from
0,E370,26,317
1,E667,196,323
2,E3190,739,468
3,E848,576,156
4,E2161,466,199


In [207]:
# Load the Cora dataset
dataset = Planetoid(root='/tmp/cora', name='Cora')

In [208]:
data = dataset[0]
data = T.NormalizeFeatures()(data)

In [209]:
train_rate = 0.85
val_ratio = (1-train_rate) / 3
test_ratio = (1-train_rate) / 3 * 2
data = train_test_split_edges(data.to(dev), val_ratio=val_ratio, test_ratio=test_ratio)

C:\Users\caleb\anaconda3\envs\pytorch\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [298]:
#For example:
dss_num = 1
Content = datasets[dss[dss_num]]['content']
Train = datasets[dss[dss_num]]['train']
Test = datasets[dss[dss_num]]['test']
Upload = datasets[dss[dss_num]]['upload']

In [299]:
Train[Train['label']==1]

,id,to,from,label
0,E5138,1747,3160,1
3,E4502,2879,2138,1
6,E365,1051,1030,1
8,E6881,1976,1071,1
13,E2263,1674,2032,1
...,...,...,...,...
7536,E1115,2515,2359,1
7537,E7112,1056,1325,1
7538,E1399,740,115,1
7541,E9426,2958,2884,1


In [300]:
def load_data(content,train, test):
    G = nx.DiGraph()
    G_test = nx.DiGraph()
    # for easier split the edges, create 2 graph, 1 with positive edge, the other with given negative edges
    G_pos = nx.DiGraph()
    G_neg = nx.DiGraph()
    graph_node_features_dict = dict()
    
    for i in range(len(content)):
        #graph_node_features_dict[content.iloc[i,0]] = np.array(content.iloc[i,1:])
        G.add_node(int(content.iloc[i,0]),features = np.array(content.iloc[i,1:]))
        G_test.add_node(int(content.iloc[i,0]),features = np.array(content.iloc[i,1:]))
        # pos and neg
        G_pos.add_node(int(content.iloc[i,0]),features = np.array(content.iloc[i,1:]))
        G_neg.add_node(int(content.iloc[i,0]),features = np.array(content.iloc[i,1:]))
        
    for i in range(len(train)):
        # Adding nodes into G
        '''
        if train.loc[i,'from'] not in G:
            G.add_node(train.loc[i,'from'],features = graph_node_features_dict[train.loc[i,'from']])
        if train.loc[i,'to'] not in G:
            G.add_node(train.loc[i,'to'],features = graph_node_features_dict[train.loc[i,'to']])
        ''' 
        # Adding edges
        G.add_edge(train.loc[i,'from'],train.loc[i,'to'],label = train.loc[i,'label'])
        
        # pos and neg
        if train.loc[i,'label'] == 0: 
            G_neg.add_edge(train.loc[i,'from'],train.loc[i,'to'],label = train.loc[i,'label'])
        else:
            G_pos.add_edge(train.loc[i,'from'],train.loc[i,'to'],label = train.loc[i,'label'])
    
    for i in range(len(test)):
        # Adding edges
        G_test.add_edge(test.loc[i,'from'],test.loc[i,'to'])
    
    adj = nx.adjacency_matrix(G,sorted(G.nodes()))
    adj_test = nx.adjacency_matrix(G_test,G_test.nodes())
    adj_pos = nx.adjacency_matrix(G_pos,sorted(G_pos.nodes()))
    adj_neg = nx.adjacency_matrix(G_neg,sorted(G_neg.nodes()))
    features = np.array(
        [features for _, features in sorted(G.nodes(data='features'), key=lambda x: x[0])])
    
    # Skip train,valid,and test mask
    
    num_features = features.shape[1]
    features = torch.FloatTensor(features)
    return adj, adj_test, adj_pos, adj_neg, features, num_features

In [301]:
g, gtest, pos, neg, features, num_features = load_data(Content,Train,Test)
A = g.toarray()
edge_index,_ = dense_to_sparse(torch.tensor(A))
Atest = gtest.toarray()
edge_index_test,_ = dense_to_sparse(torch.tensor(Atest))
Apos = pos.toarray()
edge_index_pos,_ = dense_to_sparse(torch.tensor(Apos))
Aneg = neg.toarray()
edge_index_neg,_ = dense_to_sparse(torch.tensor(Aneg))
data = Data(edge_index=edge_index_pos,x=features.to(torch.float),test = torch.tensor([Test['from'],Test['to']]))

In [302]:
train_rate = 0.9
val_ratio = (1-train_rate)#/3
test_ratio = 0#(1-train_rate) / 3 * 2
data = train_test_split_edges(data.to(dev), val_ratio=val_ratio, test_ratio=0)
               #,train_pos_edge_index = edge_index_pos,neg = edge_index_neg,test = edge_index_test)
data = T.NormalizeFeatures()(data)
data

C:\Users\caleb\anaconda3\envs\pytorch\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Data(x=[3312, 3703], test=[2, 1886], val_pos_edge_index=[2, 177], test_pos_edge_index=[2, 0], train_pos_edge_index=[2, 3202], train_neg_adj_mask=[3312, 3312], val_neg_edge_index=[2, 177], test_neg_edge_index=[2, 0])

In [303]:
model_used = 'VGNAE'
scaling_factor = 1.8
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, edge_index):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(in_channels, out_channels)
        self.linear2 = nn.Linear(in_channels, out_channels)
        self.propagate = APPNP(K=1, alpha=0)

    def forward(self, x, edge_index,not_prop=0):
        if model_used  == 'GNAE':
            x = self.linear1(x)
            x = F.normalize(x,p=2,dim=1)  * scaling_factor
            x = self.propagate(x, edge_index)
            return x

        if model_used  == 'VGNAE':
            x_ = self.linear1(x)
            x_ = self.propagate(x_, edge_index)

            x = self.linear2(x)
            x = F.normalize(x,p=2,dim=1) * scaling_factor
            x = self.propagate(x, edge_index)
            return x, x_

        return x

In [304]:
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
channels = 256

In [305]:
N = int(data.x.size()[0])
model_used = 'VGNAE'
if model_used  == 'GNAE':   
    model = GAE(Encoder(data.x.size()[1], channels, data.train_pos_edge_index)).to(dev)
if model_used  == 'VGNAE':
    model = VGAE(Encoder(data.x.size()[1], channels, data.train_pos_edge_index)).to(dev)

data.train_mask = data.val_mask = data.test_mask = data.y = None
x, train_pos_edge_index = data.x.to(dev), data.train_pos_edge_index.to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [306]:
def train():
    model.train()
    optimizer.zero_grad()
    z  = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    if model_used  in ['VGNAE']:
        loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return loss

In [307]:
def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [308]:
test(data.val_pos_edge_index, data.val_neg_edge_index)

(0.6760509432155511, 0.7106720869789769)

In [309]:
epochs = 300
for epoch in range(1,epochs):
    loss = train()
    loss = float(loss)
    
    with torch.no_grad():
        #valid_pos, valid_neg = data.val_pos_edge_index, data.val_neg_edge_index
        auc, ap = test(data.val_pos_edge_index, data.val_neg_edge_index)
        print('Epoch: {:03d}, LOSS: {:.4f}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, loss, auc, ap))

Epoch: 001, LOSS: 15.0006, AUC: 0.6856, AP: 0.7187
Epoch: 002, LOSS: 14.9991, AUC: 0.6930, AP: 0.7253
Epoch: 003, LOSS: 14.6225, AUC: 0.6953, AP: 0.7273
Epoch: 004, LOSS: 14.0947, AUC: 0.6969, AP: 0.7284
Epoch: 005, LOSS: 14.2779, AUC: 0.6979, AP: 0.7291
Epoch: 006, LOSS: 14.1817, AUC: 0.6991, AP: 0.7300
Epoch: 007, LOSS: 14.2699, AUC: 0.6997, AP: 0.7303
Epoch: 008, LOSS: 13.8928, AUC: 0.7005, AP: 0.7309
Epoch: 009, LOSS: 13.5797, AUC: 0.7013, AP: 0.7315
Epoch: 010, LOSS: 13.2887, AUC: 0.7020, AP: 0.7321
Epoch: 011, LOSS: 13.6428, AUC: 0.7027, AP: 0.7326
Epoch: 012, LOSS: 13.5392, AUC: 0.7034, AP: 0.7332
Epoch: 013, LOSS: 12.7460, AUC: 0.7040, AP: 0.7336
Epoch: 014, LOSS: 12.9277, AUC: 0.7047, AP: 0.7340
Epoch: 015, LOSS: 13.0611, AUC: 0.7053, AP: 0.7344
Epoch: 016, LOSS: 12.3376, AUC: 0.7057, AP: 0.7346
Epoch: 017, LOSS: 11.6822, AUC: 0.7062, AP: 0.7349
Epoch: 018, LOSS: 12.1587, AUC: 0.7071, AP: 0.7354
Epoch: 019, LOSS: 11.6547, AUC: 0.7077, AP: 0.7359
Epoch: 020, LOSS: 11.5789, AUC:

Epoch: 168, LOSS: 2.1113, AUC: 0.9409, AP: 0.9338
Epoch: 169, LOSS: 2.0991, AUC: 0.9410, AP: 0.9340
Epoch: 170, LOSS: 2.0235, AUC: 0.9415, AP: 0.9346
Epoch: 171, LOSS: 2.0662, AUC: 0.9417, AP: 0.9352
Epoch: 172, LOSS: 1.9541, AUC: 0.9418, AP: 0.9348
Epoch: 173, LOSS: 2.0060, AUC: 0.9424, AP: 0.9356
Epoch: 174, LOSS: 1.9404, AUC: 0.9425, AP: 0.9358
Epoch: 175, LOSS: 2.0180, AUC: 0.9427, AP: 0.9359
Epoch: 176, LOSS: 1.9667, AUC: 0.9433, AP: 0.9363
Epoch: 177, LOSS: 2.0038, AUC: 0.9432, AP: 0.9361
Epoch: 178, LOSS: 2.0398, AUC: 0.9434, AP: 0.9361
Epoch: 179, LOSS: 1.9157, AUC: 0.9433, AP: 0.9358
Epoch: 180, LOSS: 1.9576, AUC: 0.9435, AP: 0.9358
Epoch: 181, LOSS: 2.0142, AUC: 0.9435, AP: 0.9363
Epoch: 182, LOSS: 1.9515, AUC: 0.9439, AP: 0.9365
Epoch: 183, LOSS: 1.9670, AUC: 0.9441, AP: 0.9364
Epoch: 184, LOSS: 1.9359, AUC: 0.9441, AP: 0.9365
Epoch: 185, LOSS: 1.9098, AUC: 0.9445, AP: 0.9369
Epoch: 186, LOSS: 1.9872, AUC: 0.9444, AP: 0.9368
Epoch: 187, LOSS: 1.8830, AUC: 0.9442, AP: 0.9368


In [310]:
z = model.encode(x, data.test)
Upload['prob'] = model.decoder(z,data.test,sigmoid = True).detach().cpu().numpy()
Upload

,id,prob
0,E3064,0.972802
1,E298,0.950506
2,E3512,0.907563
3,E5670,0.999606
4,E5005,0.765722
...,...,...
1881,E9179,0.779397
1882,E5003,0.954958
1883,E5081,0.812506
1884,E4705,0.946395


In [311]:
Upload.to_csv('output/VGAE_new_'+str(dss_num+1)+'.csv',index=False)

In [312]:
test(data.val_pos_edge_index, data.val_neg_edge_index)

(0.9466628363497079, 0.9417587022445446)